In [1]:
import cv2
import collections
import numpy as np

In [2]:
# Detection confidence threshold
confThreshold =0.72
nmsThreshold= 0.2

# Store Coco Names in a list
classesFile = "ambulance/amb.names"
classNames = open(classesFile).read().strip().split('\n')

# class index for our required detection classes
required_class_index = [0]
detected_classNames = []

## Model Files
modelConfiguration = 'ambulance/amb.cfg'
modelWeigheights = 'ambulance/amb.weights'

# configure the network model
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)

# Configure the network backend
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Define random colour for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')

In [3]:
def configModifier():
    global classesFile
    global classNames
    global required_class_index
    global detected_classNames
    global modelConfiguration
    global modelWeigheights
    global net
    global colors
    global confThreshold
    
    confThreshold = 0.2
    classesFile = "density/coco.names"
    classNames = open(classesFile).read().strip().split('\n')
    required_class_index = [2, 3, 5, 7]
    detected_classNames = []
    modelConfiguration = 'density/yolov3.cfg'
    modelWeigheights = 'density/yolov3.weights'
    net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')

In [4]:
# Function for finding the detected objects from the network output
def postProcess(outputs,img):
    global detected_classNames 
    height, width = img.shape[:2]
    boxes = []
    classIds = []
    confidence_scores = []
    detection = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if classId in required_class_index:
                if confidence > confThreshold:
                    # print(classId)
                    w,h = int(det[2]*width) , int(det[3]*height)
                    x,y = int((det[0]*width)-w/2) , int((det[1]*height)-h/2)
                    boxes.append([x,y,w,h])
                    classIds.append(classId)
                    confidence_scores.append(float(confidence))

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)
    
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            # print(x,y,w,h)

            color = [int(c) for c in colors[classIds[i]]]
            name = classNames[classIds[i]]
            detected_classNames.append(name)
            # Draw classname and confidence score 
            cv2.putText(img,f'{name.upper()} {int(confidence_scores[i]*100)}%',
                      (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            # Draw bounding rectangle
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            detection.append([x, y, w, h, required_class_index.index(classIds[i])])

In [5]:
def densityCalculator(image):
    img = cv2.imread(image)
    blob = cv2.dnn.blobFromImage(img, 1 / 255, (320, 320), [0, 0, 0], 1, crop=False)

    # Set the input of the network
    net.setInput(blob)
    layersNames = net.getLayerNames()
    outputNames = [(layersNames[i - 1]) for i in net.getUnconnectedOutLayers()]
    # Feed data to the network
    outputs = net.forward(outputNames)

    # Find the objects from the network output
    postProcess(outputs,img)
    
    # count the frequency of detected classes
    frequency = collections.Counter(detected_classNames)
    print(frequency)
    
    #Green Signal Time Calculation
    speeds = {'car':2.25, 'bus':1.8, 'truck':1.8, 'motorbike':2.5}
    gst = (speeds['car']*frequency['car']+speeds['bus']*frequency['bus']+speeds['truck']*frequency['truck']+speeds['motorbike']*frequency['motorbike'])/5
    print("Green Signal Time :",int(gst))

    # Draw counting texts in the frame
    cv2.putText(img, "Car:        "+str(frequency['car']), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(img, "Motorbike:  "+str(frequency['motorbike']), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(img, "Bus:        "+str(frequency['bus']), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(img, "Truck:      "+str(frequency['truck']), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imshow("image", img)
    cv2.imwrite("marked.jpg",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def detector(image):
    img = cv2.imread(image)
    blob = cv2.dnn.blobFromImage(img, 1 / 255, (320, 320), [0, 0, 0], 1, crop=False)

    # Set the input of the network
    net.setInput(blob)
    layersNames = net.getLayerNames()
    outputNames = [(layersNames[i - 1]) for i in net.getUnconnectedOutLayers()]
    # Feed data to the network
    outputs = net.forward(outputNames)

    # Find the objects from the network output
    postProcess(outputs,img)

    if detected_classNames and detected_classNames[0] == 'Ambulance':
        # count the frequency of detected classes
        frequency = collections.Counter(detected_classNames)
        #print(frequency)
        print("Ambulance Detected")
        print("Lane cleared for Ambulance")
        
        # Draw counting texts in the frame
        #cv2.putText(img, "Ambulance:  "+str(frequency['Ambulance']), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow("image", img)
        cv2.imwrite("marked.jpg",img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else :
        configModifier()
        densityCalculator(image)

In [7]:
if __name__ == '__main__':
    image_path = "images/amb1.webp"
    detector(image_path)

Ambulance Detected
Lane cleared for Ambulance
